# Push-Relabel algorithm

In [ ]:
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0] * vertices for _ in range(vertices)]
        self.excess = [0] * vertices
        self.height = [0] * vertices

    def add_edge(self, u, v, w):
        self.graph[u][v] = w

def initialize_preflow(graph, source):
    """
    Initialize the preflow by setting the height of the source to the number of vertices and saturating the edges from the source.
    """
    n = graph.V
    graph.height[source] = n
    for v in range(n):
        if graph.graph[source][v] > 0:
            graph.excess[v] = graph.graph[source][v]
            graph.excess[source] -= graph.graph[source][v]
            graph.graph[v][source] = graph.graph[source][v]  # reverse edge flow
            graph.graph[source][v] = 0

def excess_nodes_exist(graph):
    """
    Check if there are any nodes with positive excess flow other than the source and sink.
    """
    for i in range(graph.V):
        if graph.excess[i] > 0 and i != source and i != sink:
            return True
    return False

def find_node_with_excess(graph):
    """
    Find a node with positive excess flow other than the source and sink.
    """
    for i in range(graph.V):
        if graph.excess[i] > 0 and i != source and i != sink:
            return i
    return None

def push(graph, u, v):
    """
    Push flow from node u to node v.
    """
    flow = min(graph.excess[u], graph.graph[u][v])
    graph.excess[u] -= flow
    graph.excess[v] += flow
    graph.graph[u][v] -= flow
    graph.graph[v][u] += flow

def discharge(graph, u):
    """
    Discharge the node u by pushing flow to its neighbors or relabeling it if necessary.
    """
    for v in range(graph.V):
        if graph.graph[u][v] > 0 and graph.height[u] > graph.height[v]:
            push(graph, u, v)
            if graph.excess[u] == 0:
                return True
    return False

def relabel(graph, u):
    """
    Relabel the node u to create an admissible edge.
    """
    min_height = float('inf')
    for v in range(graph.V):
        if graph.graph[u][v] > 0:
            min_height = min(min_height, graph.height[v])
    graph.height[u] = min_height + 1

def compute_max_flow(graph, source, sink):
    """
    Compute the maximum flow by summing the flow values in the residual graph from the source to all other nodes.
    """
    max_flow = 0
    for i in range(graph.V):
        max_flow += graph.graph[i][source]
    return max_flow

def push_relabel(graph, source, sink):
    """
    Push-Relabel algorithm to compute the maximum flow in a flow network.
    """
    initialize_preflow(graph, source)
    while excess_nodes_exist(graph):
        u = find_node_with_excess(graph)
        if not discharge(graph, u):
            relabel(graph, u)
    return compute_max_flow(graph, source, sink)

# Example usage
graph = Graph(6)
graph.add_edge(0, 1, 16)
graph.add_edge(0, 2, 13)
graph.add_edge(1, 2, 10)
graph.add_edge(1, 3, 12)
graph.add_edge(2, 1, 4)
graph.add_edge(2, 4, 14)
graph.add_edge(3, 2, 9)
graph.add_edge(3, 5, 20)
graph.add_edge(4, 3, 7)
graph.add_edge(4, 5, 4)

source = 0
sink = 5
max_flow = push_relabel(graph, source, sink)
print(f"Maximum flow: {max_flow}")

# Task Scheduling

In [ ]:
def task_scheduling(tasks, resources):
    """
    Schedules tasks to resources to maximize the total value of scheduled tasks.

    Args:
    tasks: A list of task values.
    resources: A list of available resources.

    Returns:
    The maximum total value of scheduled tasks.
    """
    dp = [0] * (len(tasks) + 1)

    # Iterate over all tasks
    for i in range(1, len(tasks) + 1):
        # Iterate over resources in reverse order
        for j in range(len(resources) - 1, -1, -1):
            # If the current resource can handle the task
            if resources[j] >= tasks[i - 1]:
                dp[i] = max(dp[i], dp[i - 1] + tasks[i - 1])
                resources[j] -= tasks[i - 1]

    return dp[-1]

# Example usage
tasks = [3, 5, 2, 7]
resources = [10, 8, 6, 12]
print(f"Maximum total value of scheduled tasks: {task_scheduling(tasks, resources)}")  # Output: 17

# Capacity Scaling

In [ ]:
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0] * vertices for _ in range(vertices)]

    def add_edge(self, u, v, w):
        self.graph[u][v] = w

def max_capacity(graph):
    """
    Finds the maximum capacity in the graph.
    """
    max_cap = 0
    for u in range(graph.V):
        for v in range(graph.V):
            max_cap = max(max_cap, graph.graph[u][v])
    return max_cap

def bfs(graph, source, sink, scaling_factor):
    """
    Breadth-First Search to find an augmenting path with sufficient residual capacity.
    """
    visited = [False] * graph.V
    parent = [-1] * graph.V
    queue = []
    queue.append(source)
    visited[source] = True

    while queue:
        u = queue.pop(0)
        for v in range(graph.V):
            if not visited[v] and graph.graph[u][v] >= scaling_factor:
                queue.append(v)
                visited[v] = True
                parent[v] = u
                if v == sink:
                    path = []
                    while v != source:
                        path.insert(0, (parent[v], v))
                        v = parent[v]
                    return path
    return None

def update_flow(graph, path, flow):
    """
    Updates the flow along the augmenting path.
    """
    for u, v in path:
        graph.graph[u][v] -= flow
        graph.graph[v][u] += flow

def capacity_scaling(graph, source, sink):
    """
    Capacity Scaling algorithm to compute the maximum flow in a flow network.
    """
    # Initialize scaling factor
    scaling_factor = max_capacity(graph)
    max_flow = 0

    while scaling_factor > 0:
        # Find augmenting paths with sufficient residual capacity
        path = bfs(graph, source, sink, scaling_factor)
        while path:
            # Update flow along the augmenting path
            update_flow(graph, path, scaling_factor)
            max_flow += scaling_factor
            # Find next augmenting path
            path = bfs(graph, source, sink, scaling_factor)
        # Reduce scaling factor
        scaling_factor //= 2

    return max_flow

# Example usage
graph = Graph(6)
graph.add_edge(0, 1, 16)
graph.add_edge(0, 2, 13)
graph.add_edge(1, 2, 10)
graph.add_edge(1, 3, 12)
graph.add_edge(2, 1, 4)
graph.add_edge(2, 4, 14)
graph.add_edge(3, 2, 9)
graph.add_edge(3, 5, 20)
graph.add_edge(4, 3, 7)
graph.add_edge(4, 5, 4)

source = 0
sink = 5
max_flow = capacity_scaling(graph, source, sink)
print(f"Maximum flow: {max_flow}")

# Dynamic Maximum Flow

In [1]:
class DynamicMaxFlow:
    def __init__(self, graph):
        """
        Initializes the DynamicMaxFlow class with a given graph.

        Args:
        graph: A 2D list representing the initial capacity matrix of the graph.
        """
        self.graph = graph
        self.flow = [[0] * len(graph) for _ in range(len(graph))]
        self.excess = [0] * len(graph)
        self.height = [0] * len(graph)

    def update_edge_capacity(self, u, v, capacity):
        """
        Updates the capacity of edge (u, v) to the specified capacity.

        Args:
        u: The starting vertex of the edge.
        v: The ending vertex of the edge.
        capacity: The new capacity of the edge.
        """
        self.graph[u][v] = capacity

    def update_edge_flow(self, u, v, flow):
        """
        Updates the flow along edge (u, v) to the specified flow.

        Args:
        u: The starting vertex of the edge.
        v: The ending vertex of the edge.
        flow: The new flow along the edge.
        """
        self.flow[u][v] = flow

    def initialize_preflow(self, source):
        """
        Initializes the preflow by setting the height of the source to the number of vertices and saturating the edges from the source.

        Args:
        source: The source vertex.
        """
        n = len(self.graph)
        self.height[source] = n
        for v in range(n):
            if self.graph[source][v] > 0:
                self.flow[source][v] = self.graph[source][v]
                self.flow[v][source] = -self.graph[source][v]
                self.excess[v] = self.graph[source][v]
                self.excess[source] -= self.graph[source][v]

    def push(self, u, v):
        """
        Pushes flow from node u to node v.

        Args:
        u: The starting vertex of the push.
        v: The ending vertex of the push.
        """
        flow = min(self.excess[u], self.graph[u][v] - self.flow[u][v])
        self.flow[u][v] += flow
        self.flow[v][u] -= flow
        self.excess[u] -= flow
        self.excess[v] += flow

    def relabel(self, u):
        """
        Relabels node u to create an admissible edge.

        Args:
        u: The vertex to be relabeled.
        """
        min_height = float('inf')
        for v in range(len(self.graph)):
            if self.graph[u][v] - self.flow[u][v] > 0:
                min_height = min(min_height, self.height[v])
        self.height[u] = min_height + 1

    def discharge(self, u):
        """
        Discharges node u by pushing flow to its neighbors or relabeling it if necessary.

        Args:
        u: The vertex to be discharged.
        """
        while self.excess[u] > 0:
            for v in range(len(self.graph)):
                if self.graph[u][v] - self.flow[u][v] > 0 and self.height[u] == self.height[v] + 1:
                    self.push(u, v)
                    if self.excess[u] == 0:
                        return
            self.relabel(u)

    def max_flow(self, source, sink):
        """
        Computes the maximum flow from source to sink using the push-relabel method.

        Args:
        source: The source vertex.
        sink: The sink vertex.

        Returns:
        The maximum flow from source to sink.
        """
        self.initialize_preflow(source)
        active = [i for i in range(len(self.graph)) if i != source and i != sink and self.excess[i] > 0]

        while active:
            u = active.pop(0)
            old_height = self.height[u]
            self.discharge(u)
            if self.height[u] > old_height:
                active.insert(0, u)

        return sum(self.flow[source][i] for i in range(len(self.graph)))

# Example usage
graph = [
    [0, 16, 13, 0, 0, 0],
    [0, 0, 10, 12, 0, 0],
    [0, 4, 0, 0, 14, 0],
    [0, 0, 9, 0, 0, 20],
    [0, 0, 0, 7, 0, 4],
    [0, 0, 0, 0, 0, 0]
]

dmf = DynamicMaxFlow(graph)
source = 0
sink = 5
print(f"Initial Maximum Flow: {dmf.max_flow(source, sink)}")

# Update the capacity of an edge
dmf.update_edge_capacity(0, 2, 20)
print(f"Updated Maximum Flow: {dmf.max_flow(source, sink)}")

Initial Maximum Flow: 29
Updated Maximum Flow: 15


# Hybrid Algorithm

In [ ]:
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0] * vertices for _ in range(vertices)]

    def add_edge(self, u, v, w):
        self.graph[u][v] = w

def compute_dp_values():
    """
    Example function to compute DP values.
    In a real scenario, this would be replaced with the actual DP logic.

    Returns:
    A list of DP values to be used in the network flow computation.
    """
    # Placeholder implementation of DP computation
    # Replace with actual DP logic
    dp_values = [1, 2, 3, 4, 5]
    return dp_values

def bfs(graph, source, sink, parent):
    """
    Breadth-First Search to find if there is a path from source to sink.

    Args:
    graph: The capacity graph.
    source: The source vertex.
    sink: The sink vertex.
    parent: The parent array to store the path.

    Returns:
    True if there is a path from source to sink, False otherwise.
    """
    visited = [False] * len(graph)
    queue = [source]
    visited[source] = True

    while queue:
        u = queue.pop(0)
        for v, capacity in enumerate(graph[u]):
            if not visited[v] and capacity > 0:
                queue.append(v)
                visited[v] = True
                parent[v] = u
                if v == sink:
                    return True
    return False

def edmonds_karp(graph, source, sink):
    """
    Edmonds-Karp algorithm to compute the maximum flow from source to sink.

    Args:
    graph: The capacity graph.
    source: The source vertex.
    sink: The sink vertex.

    Returns:
    The maximum flow value.
    """
    parent = [-1] * len(graph)
    max_flow = 0

    while bfs(graph, source, sink, parent):
        path_flow = float('Inf')
        s = sink

        while s != source:
            path_flow = min(path_flow, graph[parent[s]][s])
            s = parent[s]

        max_flow += path_flow
        v = sink

        while v != source:
            u = parent[v]
            graph[u][v] -= path_flow
            graph[v][u] += path_flow
            v = parent[v]

    return max_flow

def compute_max_flow(dp_values):
    """
    Constructs a graph based on DP values and computes the maximum flow.

    Args:
    dp_values: The DP values used to construct the graph.

    Returns:
    The maximum flow in the constructed graph.
    """
    n = len(dp_values)
    graph = Graph(n + 2)
    source = 0
    sink = n + 1

    # Example construction of a graph based on DP values
    # Replace with actual logic to construct the graph based on DP values
    for i in range(1, n + 1):
        graph.add_edge(source, i, dp_values[i - 1])
        graph.add_edge(i, sink, dp_values[i - 1])

    return edmonds_karp(graph.graph, source, sink)

def hybrid_algorithm():
    """
    Hybrid algorithm combining Dynamic Programming and Network Flow.

    Returns:
    The maximum flow computed using the hybrid algorithm.
    """
    # Dynamic Programming step
    dp_values = compute_dp_values()

    # Network Flow step
    max_flow = compute_max_flow(dp_values)

    return max_flow

# Example usage
result = hybrid_algorithm()
print(f"Maximum Flow using Hybrid Algorithm: {result}")

# Transportation Planning

In [ ]:
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0] * vertices for _ in range(vertices)]

    def add_edge(self, u, v, w):
        self.graph[u][v] = w

def compute_optimal_routes():
    """
    Example function to compute optimal routes using dynamic programming.
    In a real scenario, this would be replaced with the actual DP logic.

    Returns:
    A list of optimal routes to be used in resource allocation.
    """
    # Placeholder implementation of DP computation
    # Replace with actual DP logic
    optimal_routes = [5, 10, 15, 20]
    return optimal_routes

def bfs(graph, source, sink, parent):
    """
    Breadth-First Search to find if there is a path from source to sink.

    Args:
    graph: The capacity graph.
    source: The source vertex.
    sink: The sink vertex.
    parent: The parent array to store the path.

    Returns:
    True if there is a path from source to sink, False otherwise.
    """
    visited = [False] * len(graph)
    queue = [source]
    visited[source] = True

    while queue:
        u = queue.pop(0)
        for v, capacity in enumerate(graph[u]):
            if not visited[v] and capacity > 0:
                queue.append(v)
                visited[v] = True
                parent[v] = u
                if v == sink:
                    return True
    return False

def edmonds_karp(graph, source, sink):
    """
    Edmonds-Karp algorithm to compute the maximum flow from source to sink.

    Args:
    graph: The capacity graph.
    source: The source vertex.
    sink: The sink vertex.

    Returns:
    The maximum flow value.
    """
    parent = [-1] * len(graph)
    max_flow = 0

    while bfs(graph, source, sink, parent):
        path_flow = float('Inf')
        s = sink

        while s != source:
            path_flow = min(path_flow, graph[parent[s]][s])
            s = parent[s]

        max_flow += path_flow
        v = sink

        while v != source:
            u = parent[v]
            graph[u][v] -= path_flow
            graph[v][u] += path_flow
            v = parent[v]

    return max_flow

def allocate_resources(optimal_routes):
    """
    Constructs a graph based on optimal routes and allocates resources using network flow.

    Args:
    optimal_routes: The optimal routes to be used for resource allocation.

    Returns:
    The maximum resource allocation in the constructed graph.
    """
    n = len(optimal_routes)
    graph = Graph(n + 2)
    source = 0
    sink = n + 1

    # Example construction of a graph based on optimal routes
    # Replace with actual logic to construct the graph based on optimal routes
    for i in range(1, n + 1):
        graph.add_edge(source, i, optimal_routes[i - 1])
        graph.add_edge(i, sink, optimal_routes[i - 1])

    return edmonds_karp(graph.graph, source, sink)

def transportation_planning():
    """
    Transportation planning algorithm combining Dynamic Programming and Network Flow.

    Returns:
    The maximum resource allocation computed using the hybrid algorithm.
    """
    # Dynamic Programming step
    optimal_routes = compute_optimal_routes()

    # Network Flow step
    resource_allocation = allocate_resources(optimal_routes)

    return resource_allocation

# Example usage
result = transportation_planning()
print(f"Maximum Resource Allocation using Transportation Planning Algorithm: {result}")

# Longest Increasing Subsequence

In [ ]:
def longest_increasing_subsequence(nums):
    """
    Computes the length of the longest increasing subsequence in a list of numbers.

    Args:
    nums: A list of integers.

    Returns:
    The length of the longest increasing subsequence.
    """
    if not nums:
        return 0

    # Initialize the dp array with 1s since the minimum length of LIS for each element is 1
    dp = [1] * len(nums)

    # Iterate over the nums array
    for i in range(1, len(nums)):
        # For each element nums[i], compare with all previous elements nums[j]
        for j in range(i):
            if nums[i] > nums[j]:
                # If nums[i] is greater than nums[j], update dp[i] if it forms a longer subsequence
                dp[i] = max(dp[i], dp[j] + 1)

    # The length of the longest increasing subsequence will be the maximum value in dp array
    return max(dp)

# Example usage
nums = [10, 9, 2, 5, 3, 7, 101, 18]
print(f"Length of the Longest Increasing Subsequence: {longest_increasing_subsequence(nums)}")

# Ford-Fulkerson Algorithm

In [ ]:
from collections import deque

def bfs(graph, source, sink, parent):
    """
    Breadth-First Search to find a path from source to sink in the residual graph.

    Args:
    graph: The capacity graph.
    source: The source vertex.
    sink: The sink vertex.
    parent: The parent array to store the path.

    Returns:
    True if there is a path from source to sink, False otherwise.
    """
    visited = [False] * len(graph)
    queue = deque([source])
    visited[source] = True

    while queue:
        u = queue.popleft()
        for v, capacity in enumerate(graph[u]):
            if not visited[v] and capacity > 0:  # Check for unvisited nodes with positive capacity
                queue.append(v)
                visited[v] = True
                parent[v] = u
                if v == sink:
                    return True
    return False

def ford_fulkerson(graph, source, sink):
    """
    Ford-Fulkerson algorithm to compute the maximum flow from source to sink in a flow network.

    Args:
    graph: The capacity graph represented as an adjacency matrix.
    source: The source vertex.
    sink: The sink vertex.

    Returns:
    The maximum flow from source to sink.
    """
    max_flow = 0
    parent = [-1] * len(graph)

    while bfs(graph, source, sink, parent):
        # Find the maximum flow through the path found by BFS
        path_flow = float('Inf')
        s = sink
        while s != source:
            path_flow = min(path_flow, graph[parent[s]][s])
            s = parent[s]

        # Update the residual capacities of the edges and reverse edges along the path
        v = sink
        while v != source:
            u = parent[v]
            graph[u][v] -= path_flow
            graph[v][u] += path_flow
            v = parent[v]

        max_flow += path_flow

    return max_flow

# Example usage
graph = [
    [0, 16, 13, 0, 0, 0],
    [0, 0, 10, 12, 0, 0],
    [0, 4, 0, 0, 14, 0],
    [0, 0, 9, 0, 0, 20],
    [0, 0, 0, 7, 0, 4],
    [0, 0, 0, 0, 0, 0]
]

source = 0
sink = 5
print(f"Maximum Flow: {ford_fulkerson(graph, source, sink)}")
